In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [33]:
df=pd.read_csv('./mushrooms.csv')
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [34]:
le=LabelEncoder()
ds=df.apply(le.fit_transform)
ds

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


In [35]:
print(type(ds))
data=ds.values
# data=np.array(ds)
print(type(data))
data.shape

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


(8124, 23)

In [36]:
data_y=data[:,0]
data_x=data[:,1:]
print(data_x.shape,data_y.shape)

(8124, 22) (8124,)


In [37]:
x_train,x_test,y_train,y_test=train_test_split(data_x,data_y,test_size=0.2)

In [38]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(6499, 22) (6499,)
(1625, 22) (1625,)


In [39]:
def prior_probability(x_train,y_train,label_value):
    n_rows=x_train[y_train==label_value]
    prob_value=n_rows.shape[0]/float(x_train.shape[0])
    return prob_value
prior_probability(x_train,y_train,1)
prior_probability(x_train,y_train,0)

0.5140790890906294

In [40]:
def cond_probability(x_train,y_train,label_value,feature_col,feature_val):
    n_rows=x_train[y_train==label_value]
    cons_rows=n_rows[n_rows[:,feature_col]==feature_val]
    prob_value=cons_rows.shape[0]/float(n_rows.shape[0])
    return prob_value


In [41]:
classes=np.unique(y_train)
classes

array([0, 1])

In [42]:
preds=[]
c=0
for ix in range(x_test.shape[0]):
    post_prob=[]
    for cls in classes:
        likelihood=1.0
        for feature in range(x_test.shape[1]):
            cond=cond_probability(x_train,y_train,cls,feature,x_test[ix,feature])
            likelihood *=cond
        prior=prior_probability(x_train,y_train,cls)
        posterior=prior*likelihood
        post_prob.append(posterior)
    post_prob=np.array(post_prob)
    pred=post_prob.argmax()
    if(pred==y_test[ix]):  #Accuracy
        c=c+1
    preds.append(classes[pred])


In [43]:
accuracy=c/float(x_test.shape[0])
# accuracy

# preds=np.array(preds)
# accuracy=np.sum(np.array(y_test==preds))/float(y_test.shape[0])
# accuracy

0.9987692307692307

In [57]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(x_train, y_train)

neigh.score(x_test,y_test)

0.9956923076923077

In [49]:
# logistic Regression
# def  h(x,theta):
#     return 1.0/(1+np.exp(-1*np.dot(theta,x)))
# def gradients(x,y,theta):
#     grads=np.zeros((x.shape[1],))
#     for ix in range(x.shape[0]):
#         hx=h(x[ix],theta)
#         grads+=(hx-y[ix])*x[ix]
#     return grads/x.shape[0]

# def error(x, y, theta):
#     err = 0.0
#     for ix in range(x.shape[0]):
#         hx = h(x[ix], theta)
#         err += -y[ix]*np.log(hx) - (1-y[ix])*np.log(1-hx)
    
#     return err 

# def gradient_descent(x,y,lr=0.01,epochs=10):
#     theta=np.zeros((x.shape[1],))
#     error_list = []
#     for _ in range(epochs):
#         error_list.append(error(x, y, theta))
#         g=gradients(x,y,theta)
#         theta=theta-lr*g
#     return theta,error_list

In [59]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(x_train, y_train)
# print(clf.coef_, clf.intercept_)

clf.score(x_test, y_test)

C:\Users\abhishek\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9587692307692308

In [60]:
from sklearn import svm
sv = svm.SVC(kernel='linear')
sv.fit(x_train, y_train)

sv.score(x_train, y_train)

0.9804585320818587